# A Manual Regristation Pipeline for Processing

In [1]:
import numpy as np
import copy
import open3d as o3d
import os
import sys

from numpy.linalg import inv

from scipy.spatial import cKDTree
import copy
import teaserpp_python

# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
#import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
#o3dtut.interactive = not "CI" in os.environ

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def extract_fpfh(pcd, voxel_size):
  radius_normal = voxel_size * 2
  pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

  radius_feature = voxel_size * 5
  fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
  return np.array(fpfh.data).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
  feat1tree = cKDTree(feat1)
  dists, nn_inds = feat1tree.query(feat0, k=knn, n_jobs=-1)
  if return_distance:
    return nn_inds, dists
  else:
    return nn_inds

def find_correspondences(feats0, feats1, mutual_filter=True):
  nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
  corres01_idx0 = np.arange(len(nns01))
  corres01_idx1 = nns01

  if not mutual_filter:
    return corres01_idx0, corres01_idx1

  nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
  corres10_idx1 = np.arange(len(nns10))
  corres10_idx0 = nns10

  mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
  corres_idx0 = corres01_idx0[mutual_filter]
  corres_idx1 = corres01_idx1[mutual_filter]

  return corres_idx0, corres_idx1

def get_teaser_solver(noise_bound):
    solver_params = teaserpp_python.RobustRegistrationSolver.Params()
    solver_params.cbar2 = 1.0
    solver_params.noise_bound = noise_bound
    solver_params.estimate_scaling = False
    solver_params.inlier_selection_mode = \
        teaserpp_python.RobustRegistrationSolver.INLIER_SELECTION_MODE.PMC_EXACT
    solver_params.rotation_tim_graph = \
        teaserpp_python.RobustRegistrationSolver.INLIER_GRAPH_FORMULATION.CHAIN
    solver_params.rotation_estimation_algorithm = \
        teaserpp_python.RobustRegistrationSolver.ROTATION_ESTIMATION_ALGORITHM.GNC_TLS
    solver_params.rotation_gnc_factor = 1.4
    solver_params.rotation_max_iterations = 10000
    solver_params.rotation_cost_threshold = 1e-16
    solver = teaserpp_python.RobustRegistrationSolver(solver_params)
    return solver

def Rt2T(R,t):
    T = np.identity(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T 

In [40]:
# Camera views
# -- 11 is uptop front - 0 (cloud_0)
# -- 12 is left bottom - 1
# -- 13 is right bottom - 2
# -- 14 is right top view - 3
# -- 15 is uptop back -  4
# -- 17 is left top view - 5

# Pairs should be
# 12,17 (1,5) - left_up_down
# 13,14 (2,3) - right_up_down
# 11,15 (0,4) - uptop_back_front

In [58]:
A_pcd = copy.deepcopy(A_pcd_raw).transform(T_icp)
pcd23 = o3d.geometry.PointCloud()
pcd23 = A_pcd + B_pcd_raw
o3d.visualization.draw_geometries([pcd23])

In [56]:
A_pcd = copy.deepcopy(A_pcd_raw).transform(T_icp)
pcd04 = o3d.geometry.PointCloud()
pcd04 = A_pcd + B_pcd_raw
o3d.visualization.draw_geometries([pcd04])

In [54]:
A_pcd = copy.deepcopy(A_pcd_raw).transform(T_icp)
pcd15 = o3d.geometry.PointCloud()
pcd15 = A_pcd + B_pcd_raw
o3d.visualization.draw_geometries([pcd15])

In [60]:
A_pcd = copy.deepcopy(A_pcd_raw).transform(T_icp)
pcd2304 = o3d.geometry.PointCloud()
pcd2304 = A_pcd + B_pcd_raw
o3d.visualization.draw_geometries([pcd2304])

In [62]:
A_pcd = copy.deepcopy(A_pcd_raw).transform(T_icp)
pcd1504 = o3d.geometry.PointCloud()
pcd1504 = A_pcd + B_pcd_raw
o3d.visualization.draw_geometries([pcd1504])

In [43]:
H_23_04 = T_icp

In [46]:
H_15_04 = T_icp

In [63]:
o3d.visualization.draw_geometries([pcd1504,pcd2304])

In [66]:
o3d.io.write_point_cloud("firstOne_all_views.ply", firstOne)

True

In [51]:
pwd

'/home/goowfd/SoftWare/Register_cattle_jupyter'

In [65]:
A_pcd = copy.deepcopy(A_pcd_raw).transform(T_icp)
firstOne = o3d.geometry.PointCloud()
firstOne = A_pcd + B_pcd_raw
o3d.visualization.draw_geometries([firstOne])

In [5]:
# Pairs should be
# 12,17 (1,5) - left_up_down
# 13,14 (2,3) - right_up_down
# 11,15 (0,4) - uptop_back_front

filepath = "/home/goowfd/DataSets/SecondOne_PLY2/"
# Load and visualize two point clouds from 3DMatch dataset

pcd2 = o3d.io.read_point_cloud(filepath+"cloud_2.ply")
pcd3 = o3d.io.read_point_cloud(filepath+"cloud_3.ply")

pcd1 = o3d.io.read_point_cloud(filepath+"cloud_1.ply")
pcd5 = o3d.io.read_point_cloud(filepath+"cloud_5.ply")

pcd0 = o3d.io.read_point_cloud(filepath+"cloud_0.ply")
pcd4 = o3d.io.read_point_cloud(filepath+"cloud_4.ply")

res23 = register_two_views_teaser(pcd2,pcd3)
res15 = register_two_views_teaser(pcd1,pcd5)
res04 = register_two_views_teaser(pcd0,pcd4)

res2304 = register_two_views_teaser(res23,res04)
res1504 = register_two_views_teaser(res15,res04)
all_views = register_two_views_teaser(res2304,res1504)




/tmp/ipykernel_5619/714796952.py:16: DeprecationWarning: The n_jobs argument has been renamed "workers". The old name "n_jobs" will stop working in SciPy 1.8.0.
  dists, nn_inds = feat1tree.query(feat0, k=knn, n_jobs=-1)


FPFH generates 668 putative correspondences.
Starting scale solver.
Scale estimation complete.
Max core number: 44
Num vertices: 669
*** [pmc heuristic: thread 3]   current max clique = 27,  time = 0.000302076 sec
*** [pmc heuristic: thread 5]   current max clique = 35,  time = 0.000324011 sec
*** [pmc heuristic: thread 1]   current max clique = 36,  time = 0.000330925 sec
[pmc heuristic]	 mc = 36
Created adjacency matrix in 0.000136852 seconds
[pmc: initial k-core pruning]  before pruning: |V| = 668, |E| = 17702
[pmc: initial k-core pruning]  after pruning:  |V| = 76, |E| = 2145
[pmc]  initial pruning took 4.41074e-05 sec
[pmc: sorting neighbors]  |E| = 4290, |E_sorted| = 4290
|V| = 73
*** [pmc: thread 1]   current max clique = 37,  time = 0.000315189 sec
-----------------------------------------------------------------------
Max Clique of scale estimation inliers: 
Using chain graph for GNC rotation.
Starting rotation solver.
GNC rotation estimation noise bound:0.1
GNC rotation estim

Failed to establish dbus connection

FPFH generates 941 putative correspondences.
Starting scale solver.
Scale estimation complete.
Max core number: 69
Num vertices: 942
*** [pmc heuristic: thread 1]   current max clique = 58,  time = 5.19753e-05 sec
*** [pmc heuristic: thread 10]   current max clique = 59,  time = 6.8903e-05 sec
[pmc heuristic]	 mc = 59
Created adjacency matrix in 0.00022006 seconds
[pmc: initial k-core pruning]  before pruning: |V| = 941, |E| = 29095
[pmc: initial k-core pruning]  after pruning:  |V| = 941, |E| = 4337
[pmc]  initial pruning took 8.39233e-05 sec
[pmc: sorting neighbors]  |E| = 8674, |E_sorted| = 8674
|V| = 100
*** [pmc: thread 11]   current max clique = 60,  time = 0.000649929 sec
-----------------------------------------------------------------------
Max Clique of scale estimation inliers: 
Using chain graph for GNC rotation.
Starting rotation solver.
GNC rotation estimation noise bound:0.1
GNC rotation estimation noise bound squared:0.01
GNC-TLS solver terminated due to cost convergenc

In [4]:
def register_two_views_teaser(A_pcd_raw,B_pcd_raw):
    VOXEL_SIZE = 0.05
    VISUALIZE = False

    #A_pcd_raw.paint_uniform_color([0.0, 0.0, 1.0]) # show A_pcd in blue
    #B_pcd_raw.paint_uniform_color([1.0, 0.0, 0.0]) # show B_pcd in red
    if VISUALIZE:
        o3d.visualization.draw_geometries([A_pcd_raw,B_pcd_raw]) # plot A and B 

    # voxel downsample both clouds
    A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    if VISUALIZE:
        o3d.visualization.draw_geometries([A_pcd,B_pcd]) # plot downsampled A and B 

    A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
    B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

    # extract FPFH features
    A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
    B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)

    # establish correspondences by nearest neighbour search in feature space
    corrs_A, corrs_B = find_correspondences(
        A_feats, B_feats, mutual_filter=True)
    A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
    B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

    num_corrs = A_corr.shape[1]
    print(f'FPFH generates {num_corrs} putative correspondences.')

    # visualize the point clouds together with feature correspondences
    points = np.concatenate((A_corr.T,B_corr.T),axis=0)
    lines = []
    for i in range(num_corrs):
        lines.append([i,i+num_corrs])
    colors = [[0, 1, 0] for i in range(len(lines))] # lines are shown in green
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    #o3d.visualization.draw_geometries([A_pcd,B_pcd,line_set])

    # robust global registration using TEASER++
    NOISE_BOUND = VOXEL_SIZE
    teaser_solver = get_teaser_solver(NOISE_BOUND)
    teaser_solver.solve(A_corr,B_corr)
    solution = teaser_solver.getSolution()
    R_teaser = solution.rotation
    t_teaser = solution.translation
    T_teaser = Rt2T(R_teaser,t_teaser)

    # Visualize the registration results
    A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
    #o3d.visualization.draw_geometries([A_pcd_T_teaser,B_pcd])

    # local refinement using ICP
    icp_sol = o3d.pipelines.registration.registration_icp(
          A_pcd, B_pcd, NOISE_BOUND, T_teaser,
          o3d.pipelines.registration.TransformationEstimationPointToPoint(),
          o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
    T_icp = icp_sol.transformation

    # visualize the registration after ICP refinement
    A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
    o3d.visualization.draw_geometries([A_pcd_T_icp,B_pcd])
    tformed_A = copy.deepcopy(A_pcd_raw).transform(T_icp)
    res = o3d.geometry.PointCloud()
    res = tformed_A + B_pcd_raw
    
    return res

Starting scale solver.
Scale estimation complete.
Max core number: 76
Num vertices: 719
*** [pmc heuristic: thread 9]   current max clique = 68,  time = 5.50747e-05 sec
*** [pmc heuristic: thread 1]   current max clique = 69,  time = 0.00370502 sec
*** [pmc heuristic: thread 3]   current max clique = 70,  time = 0.00371599 sec
[pmc heuristic]	 mc = 70
Created adjacency matrix in 0.000152111 seconds
[pmc: initial k-core pruning]  before pruning: |V| = 718, |E| = 24125
[pmc: initial k-core pruning]  after pruning:  |V| = 93, |E| = 3952
[pmc]  initial pruning took 6.8903e-05 sec
[pmc: sorting neighbors]  |E| = 7904, |E_sorted| = 7904
|V| = 91
-----------------------------------------------------------------------
Max Clique of scale estimation inliers: 
Using chain graph for GNC rotation.
Starting rotation solver.
GNC rotation estimation noise bound:0.1
GNC rotation estimation noise bound squared:0.01
GNC-TLS solver terminated due to cost convergence.
Cost diff: 0
Iterations: 12
Rotation 

## Utility Functions

In [2]:
def demo_crop_geometry():
    print("Demo for manual geometry cropping")
    print(
        "1) Press 'Y' twice to align geometry with negative direction of y-axis"
    )
    print("2) Press 'K' to lock screen and to switch to selection mode")
    print("3) Drag for rectangle selection,")
    print("   or use ctrl + left click for polygon selection")
    print("4) Press 'C' to get a selected geometry and to save it")
    print("5) Press 'F' to switch to freeview mode")
    pcd = o3d.io.read_point_cloud("../../test_data/ICP/cloud_bin_0.pcd")
    o3d.visualization.draw_geometries_with_editing([pcd])

def draw_registration_color(source,target,transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])    

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def pick_points(pcd, filename, fileid):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.capture_screen_image("results/%s_%d.png" % (filename, fileid))    
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

## Pair-Wise Registration
### left view
|Source|Target|
|---|---|
|1|2|
|1|3|
|3|4|

### right view
|Source|Target|
|---|---|
|5|6|
|5|7|
|7|8|

### top view
|Source|Target|
|---|---|
|9|1|
|9|5|

### Suggested order:

left12, left13, left34, right56, right57, right78, top91, top95

In [5]:
#filepath = "/home/iris/yg5d6/Dropbox/Thesis/Open3D/examples/python/reconstruction_system/dataset_cow/raw/"
filepath = "/home/goowfd/DataSets/FirstOne_Plys/"

exp_name = 'top01'
source_id = 0
target_id = 1

source = o3d.io.read_point_cloud("%s/cloud_%d.ply" % (filepath, source_id))
target = o3d.io.read_point_cloud("%s/cloud_%d.ply" % (filepath, target_id))
print("Visualization of two point clouds before manual alignment")
draw_registration_color(source, target, np.identity(4))

Visualization of two point clouds before manual alignment


In [7]:
o3d.visualization.draw_geometries([target])

In [8]:
# pick points from two point clouds and builds correspondences
picked_id_source = pick_points(source, exp_name, source_id)
picked_id_target = pick_points(target, exp_name, target_id)

assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
assert (len(picked_id_source) == len(picked_id_target))

# corr contains the correspondence points
# TO-DO: Save the correspondence points for future conveniences
corr = np.zeros((len(picked_id_source), 2))
corr[:, 0] = picked_id_source
corr[:, 1] = picked_id_target


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #55407 (-0.14, 0.042, 0.76) to add in queue.
[Open3D INFO] Picked point #139023 (0.27, 0.39, 1.5) to add in queue.
[Open3D INFO] Picked point #107816 (-1.1, -1.0, 1.9) to add in queue.
[Open3D INFO] Picked point #42455 (0.25, -0.42, 1.4) to add in queue.
[Open3D WARNING] Write PNG failed: unable to write file: results/top01_0.png


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #40471 (0.64, -0.65, 1.3) to add in queue.
[Open3D INFO] Picked point #27938 (0.78, -0.12, 1.0) to add in queue.
[Open3D INFO] Picked point #87973 (0.015, 0.55, 2.4) to add in queue.
[Open3D INFO] Picked point #12424 (1.3, -0.044, 1.8) to add in que

In [9]:
# estimate rough transformation using correspondences
print("Compute a rough transform using the correspondences given by user")
p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
trans_init = p2p.compute_transformation(source, target,
                                        o3d.utility.Vector2iVector(corr))
print(trans_init)

Compute a rough transform using the correspondences given by user
[[ 0.97510203 -0.22023404  0.02594229  0.76280059]
 [-0.0521832  -0.11418264  0.99208832 -1.48437336]
 [-0.21552946 -0.96874109 -0.12283224  1.5170968 ]
 [ 0.          0.          0.          1.        ]]


In [10]:
# point-to-point ICP for refinement
print("Perform point-to-point ICP refinement")
threshold = 0.03  # 3cm distance threshold
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p.transformation)

Perform point-to-point ICP refinement
[[ 0.97109404 -0.23441209  0.04502595  0.73704117]
 [-0.07444678 -0.11821238  0.99019367 -1.48699749]
 [-0.22679075 -0.96492321 -0.13224656  1.55066191]
 [ 0.          0.          0.          1.        ]]


In [11]:
draw_registration_result(source, target, reg_p2p.transformation)
# draw_registration_color(source, target, reg_p2p.transformation)
print("")

In [75]:
# Save the parameters
np.savetxt('results/%s_corr.txt' % (exp_name), corr, delimiter=',')
np.savetxt('results/%s_transform.txt' % (exp_name), reg_p2p.transformation, delimiter=',')

## Register all Point Clouds (Left, Right, Top)

In [10]:
# Load all point clouds
source  = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 9))

left1   = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 1))
left2   = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 2))
left3   = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 3))
# left4   = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 4))

right5 = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 5))
right6 = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 6))
right7 = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 7))
right8 = o3d.io.read_point_cloud("%s/depthImg_001%d.ply" % (filepath, 8))

# Load all transformations
H91 = np.loadtxt('/home/goowfd/DataSets/cow_data/htms/top91_transform.txt', delimiter=',')
H95 = np.loadtxt('results/top95_transform.txt', delimiter=',')

H12 = np.loadtxt('results/left12_transform.txt', delimiter=',')
H13 = np.loadtxt('results/left13_transform.txt', delimiter=',')
# H34 = np.loadtxt('results/right78_transform.txt', delimiter=',')

H56 = np.loadtxt('results/right56_transform.txt', delimiter=',')
H57 = np.loadtxt('results/right57_transform.txt', delimiter=',')
H78 = np.loadtxt('results/right78_transform.txt', delimiter=',')

OSError: results/right78_transform.txt not found.

In [8]:
# Register with colored point clouds
source_temp = copy.deepcopy(source)

left1_temp = copy.deepcopy(left1)
left2_temp = copy.deepcopy(left2)
left3_temp = copy.deepcopy(left3)
# left4_temp = copy.deepcopy(left4)

right5_temp = copy.deepcopy(right5)
right6_temp = copy.deepcopy(right6)
right7_temp = copy.deepcopy(right7)
right8_temp = copy.deepcopy(right8)

source_temp.paint_uniform_color([1, 0.7, 0])

left1_temp.paint_uniform_color([0, 0.6, 0.9])
left2_temp.paint_uniform_color([0, 0.6, 0.9])
left3_temp.paint_uniform_color([0, 0.6, 0.9])
# left4_temp.paint_uniform_color([0, 0.6, 0.9])

right5_temp.paint_uniform_color([0.9, 0, 0.2])
right6_temp.paint_uniform_color([0.9, 0, 0.2])
right7_temp.paint_uniform_color([0.9, 0, 0.2])
right8_temp.paint_uniform_color([0.9, 0, 0.2])

# target3_temp.paint_uniform_color([0.5, 0, 1])

left1_temp.transform(inv(H91))
left2_temp.transform(inv(H91) @ inv(H12))
left3_temp.transform(inv(H91) @ inv(H13))
# left4_temp.transform(inv(H91) @ inv(H13) @ inv(H34))

right5_temp.transform(inv(H95))
right6_temp.transform(inv(H95) @ inv(H56))
right7_temp.transform(inv(H95) @ inv(H57))
right8_temp.transform(inv(H95) @ inv(H57) @ inv(H78))

o3d.visualization.draw_geometries([source_temp, 
                                   left1_temp, left2_temp, left3_temp, 
                                   right5_temp, right6_temp, right7_temp, right8_temp])

In [12]:
h91 = inv(H91)
h92 = inv(H91) * inv(H12)
h93 = inv(H91) * inv(H13)
#h94 = inv(H91) * inv(H13) * inv(H34)


h95 = inv(H95)
h96 = inv(H95) * inv(H56) 
h97 = inv(H95) * inv(H57) 
h98 = inv(H95) * inv(H57) * inv(H78) 


left1_temp = copy.deepcopy(left1)
left2_temp = copy.deepcopy(left2)
left3_temp = copy.deepcopy(left3)
left4_temp = copy.deepcopy(left4)

right5_temp = copy.deepcopy(right5)
right6_temp = copy.deepcopy(right6)
right7_temp = copy.deepcopy(right7)
right8_temp = copy.deepcopy(right8)

left1_temp.transform(h91)
left2_temp.transform(h92)
left3_temp.transform(h93)
#left4_temp.transform(h94)

right5_temp.transform(h95)
right6_temp.transform(h97)
right7_temp.transform(h97)
right8_temp.transform(h97)


source_temp.paint_uniform_color([1, 0.7, 0])

left1_temp.paint_uniform_color([0, 0.6, 0.9])
left2_temp.paint_uniform_color([0, 0.6, 0.9])
left3_temp.paint_uniform_color([0, 0.6, 0.9])
# left4_temp.paint_uniform_color([0, 0.6, 0.9])

right5_temp.paint_uniform_color([0.9, 0, 0.2])
right6_temp.paint_uniform_color([0.9, 0, 0.2])
right7_temp.paint_uniform_color([0.9, 0, 0.2])
right8_temp.paint_uniform_color([0.9, 0, 0.2])

o3d.visualization.draw_geometries([source_temp, 
                                   left1_temp, left2_temp, left3_temp, 
                                   right5_temp, right6_temp, right7_temp, right8_temp])

In [15]:
np.savetxt('tforms_wrt9/h91_transform.txt', h91, delimiter=',')
np.savetxt('tforms_wrt9/h92_transform.txt', h92, delimiter=',')
np.savetxt('tforms_wrt9/h93_transform.txt', h93, delimiter=',')
np.savetxt('tforms_wrt9/h95_transform.txt', h95, delimiter=',')
np.savetxt('tforms_wrt9/h96_transform.txt', h96, delimiter=',')
np.savetxt('tforms_wrt9/h97_transform.txt', h97, delimiter=',')
np.savetxt('tforms_wrt9/h98_transform.txt', h98, delimiter=',')


In [13]:
@staticmethod
    def read_gt_list(filename):
        poses = np.loadtxt(filename, delimiter=",")
        return poses.reshape((len(poses), 4, 4))

'/home/goowfd/SoftWare/Register_cattle_jupyter'

In [37]:
path = '/home/goowfd/DataSets/cow_data/188_full_all_PLYS/data/poses2.txt'
poses = np.loadtxt(path,delimiter=",")
print(poses)
print("\n*********************\n")
print(len(poses))
print("\n*********************\n")
poses.reshape((1, 4, 4))

print(poses)
print(poses.shape)

[[-0.30212592 -0.06730498 -0.17576259 -1.07551608]
 [-0.12858885  0.0573488  -0.00996986  0.09521281]
 [-0.05328764  0.04730134  0.4403019   0.18700693]
 [ 0.          0.          0.          1.        ]]

*********************

4

*********************

[[-0.30212592 -0.06730498 -0.17576259 -1.07551608]
 [-0.12858885  0.0573488  -0.00996986  0.09521281]
 [-0.05328764  0.04730134  0.4403019   0.18700693]
 [ 0.          0.          0.          1.        ]]
(4, 4)


In [32]:
# Register with original point clouds
source_temp = copy.deepcopy(source)

left1_temp = copy.deepcopy(left1)
left2_temp = copy.deepcopy(left2)
left3_temp = copy.deepcopy(left3)
# left4_temp = copy.deepcopy(left4)

right5_temp = copy.deepcopy(right5)
right6_temp = copy.deepcopy(right6)
right7_temp = copy.deepcopy(right7)
right8_temp = copy.deepcopy(right8)

left1_temp.transform(inv(H91))
left2_temp.transform(inv(H91) @ inv(H12))
left3_temp.transform(inv(H91) @ inv(H13))
# left4_temp.transform(inv(H91) @ inv(H13) @ inv(H34))

right5_temp.transform(inv(H95))
right6_temp.transform(inv(H95) @ inv(H56))
right7_temp.transform(inv(H95) @ inv(H57))
right8_temp.transform(inv(H95) @ inv(H57) @ inv(H78))

o3d.visualization.draw_geometries([source_temp, 
                                   left1_temp, left2_temp, left3_temp, 
                                   right5_temp, right6_temp, right7_temp, right8_temp])